In [1]:
import cv2
import numpy as np
import cupy as cp
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime
from profiler import *

In [2]:
def pyramid(image, gradient_channels, binary_imgs, initial_scale, final_scale, scale):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in gradient_channels], [imutils.resize(np.uint8(img), width=w).astype(bool) for img in binary_imgs],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    ignored_height_ratio=0.257 #calculated in dataset_processing.ipynb
    height=image.shape[0]
    start_height=int(ignored_height_ratio*image.shape[0])
    for y in range(start_height, height, stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
def area(rect):
    ((x,y),(x2,y2))=rect
    return (x2-x)*(y2-y)

def calc_overlapping(a,b,method):
    dx = min(a[1][0], b[1][0]) - max(a[0][0], b[0][0])
    dy = min(a[1][1], b[1][1]) - max(a[0][1], b[0][1])
    basic_area=method([area(a), area(b)])
    overlap_area=dx*dy
    is_overlapping=((dx>=0) and (dy>=0))
    overlapping_ratio=overlap_area/basic_area
    
    return overlapping_ratio if is_overlapping else 0

def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

## Load classifier

In [4]:
os.chdir(PROJECT_PATH)
clf_grad = load('rand_forest_clf.joblib') 
clf_grad.set_params(n_jobs=-1)

color_clfs=[load('naive_bayes_clf_{}.joblib'.format(color)) for color in COLORS]
color_clf_map=dict(zip(COLORS, color_clfs))

## Set params

In [5]:
(winW, winH) = (WIN_W, WIN_H)
channels_no=8
DS_min_winH_m=41
DS_max_winH_m=281
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m
RESIZING_SCALE=1.15
MAX_PRED_OVERLAPPING=0.5
MIN_PRED=0.5
WHITE=(255, 255, 255)
DRAW_BORDER=4
KERNEL_W_RATIO=7.0/640.0 #because kernel (7,7) was the best for VGA images in previous research

In [6]:
def binarize(color_channels, color_clfs):
    s0, s1, s2 = color_channels[0].shape
    flat_ch = [np.reshape(img, (s0*s1, s2)) for img in color_channels]
    flat_img = np.hstack(flat_ch)
    flat_binaries=[clf.predict(flat_img) for clf in color_clfs]
    img_binaries=[np.reshape(flat, (s0, s1)) for flat in flat_binaries]
    return img_binaries

def open_bin(img, kernel):
    kernel = np.ones(kernel, np.uint8) 
    img_erosion = cv2.erode(img, kernel, iterations=1)
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    return img_dilation

def close_bin(img, kernel):
    kernel = np.ones(kernel, np.uint8) 
    img_dilation = cv2.dilate(img, kernel, iterations=1)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    return img_erosion

def morpho(img):
    h,w=img.shape
    k=int(w*KERNEL_W_RATIO)
    kernel=(k,k)
    return np.bool_(open_bin(close_bin(np.uint8(img), kernel), kernel))

h=winH/1.2
w=winW/1.2
x0=winW/12.0
y0=winH/12.0
x1=winW*11.0/12.0
y1=winH*11.0/12.0
((h1x1,h1y1),(h1x2,h1y2))=((int(0.126*w+x0), int(0.3056*h+y0)),(int(0.409*w+x0), int(h+y0)))
((h2x1,h2y1),(h2x2,h2y2))=((int(0.5906*w+x0), int(0.3056*h+y0)),(int(0.875*w+x0), int(h+y0)))

def color_score(win):
    rect=win[int(y0):int(y1),int(x0):int(x1)]
    hole_1=win[h1y1:h1y2, h1x1:h1x2]
    hole_2=win[h2y1:h2y2, h2x1:h2x2]
    
    all_points=rect.shape[0]*rect.shape[1]
    hole_1_points=hole_1.shape[0]*hole_1.shape[1]
    hole_2_points=hole_2.shape[0]*hole_2.shape[1]
    
    hole_1_positives=np.count_nonzero(hole_1)
    hole_2_positives=np.count_nonzero(hole_2)
    rect_score=np.count_nonzero(rect)-hole_1_positives-hole_2_positives
    hole_1_score=hole_1_points-hole_1_positives
    hole_2_score=hole_2_points-hole_2_positives
    score=(rect_score+hole_1_score+hole_2_score)/all_points
    
#     print("full",all_points, hole_1_points, hole_2_points)
#     print("positives",rect_score, hole_1_positives, hole_2_positives)
#     print("scores",rect_score, hole_1_score, hole_2_score)
#     print(score)
    
#     for e in [win, rect, hole_1, hole_2]:
#         cv2.imshow("Window", np.float32(e))
#         cv2.waitKey(0)
    
    return score

In [7]:
def show(img, x, y, winW, winH, positives=[]):
    clone = np.float32(img.copy())
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), WHITE, DRAW_BORDER)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), WHITE, DRAW_BORDER)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def pred_overlapping(row, scaled_rect):
    x,y,w,h = scaled_rect
    a = ((x,y),(x+w,y+h))
    [scene_name, filename, scale, (rx,ry)] = row
    [rx,ry,rw,rh] = scale_many([rx,ry,winW,winH], scale)
    b = ((rx,ry),(rx+rw,ry+rh))

    overlap=calc_overlapping(a,b,np.min)
    return overlap
    
def image_predict(vals,IDX,rects_count,color_mode):
    global glob_RES
    if len(vals)!=0 and rects_count>0:
        if color_mode:
            pred=np.array([np.max(cs) for cs in vals])
        else:
            pred=np.array([p[1] for p in clf_grad.predict_proba(vals)])
        greater_than_09=pred > MIN_PRED
        pred=pred[greater_than_09]
        IDX=IDX[greater_than_09]
        max_pred_ids=[]
        max_preds=[]
        max_pred_IDXs=[]
        for i in range(rects_count):
            if len(pred) >0:
                max_pred_id=np.argmax(pred)
                max_pred=pred[max_pred_id]
                max_preds.append(max_pred)
                max_pred_IDX=IDX[max_pred_id]
                max_pred_IDXs.append(max_pred_IDX)

                [scene_name, filename, scale, (x,y)]=max_pred_IDX
                scaled_rect=scale_many([x,y,winW,winH], scale)

                not_within_bools = np.array([pred_overlapping(row, scaled_rect) < MAX_PRED_OVERLAPPING for row in IDX])
                pred=pred[not_within_bools]
                IDX=IDX[not_within_bools]
            else:
                print("No more predictions with prob > {}".format(MIN_PRED))
        if len(max_preds)>0:
            max_preds=np.array([[e] for e in max_preds])
            res=np.append(max_pred_IDXs, max_preds, 1)
            glob_RES=np.vstack([glob_RES,res])
    
def predict(filename, scene_name, label_resolution, rects, *args):
    X=[]
    IDX=[]
    COL_SCORS=[]
    
    image=imread_resized(scene_name, filename, label_resolution)
    gradient_channels=read_split_channels(scene_name, filename, "channels_gradient")
    gradient_channels=[ch.astype('float32') for ch in gradient_channels] #if not, scikit will do it later slower
    color_channels=read_split_channels(scene_name, filename, "channels_color")
    binary_imgs=binarize(color_channels, color_clfs)
    binary_imgs=[morpho(img) for img in binary_imgs]
    for resized_img, resized_gradient_ch, resized_binary_imgs, scale in pyramid(image, gradient_channels, binary_imgs, initial_scale, final_scale, scale=RESIZING_SCALE):
        resized_ch=np.dstack(resized_gradient_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=4, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            color_ch_windows=[img[y:y + winH, x:x + winW] for img in resized_binary_imgs]
            color_scores=[color_score(w) for w in color_ch_windows]
            COL_SCORS.append(color_scores)
            color_ch_windows=None
            X.append(channels_window)
            row=[scene_name, filename, scale, (x,y)]
            IDX.append(row)
            #show(resized_binary_imgs[0], x, y, winW, winH)
    X=np.reshape(X,(len(X),winW*winH*channels_no)) #cupy does not help
#     image_predict(X,np.array(IDX), len(rects), False)
    image_predict(COL_SCORS, np.array(IDX), len(rects), True)

In [8]:
glob_RES=np.empty((0,5), object)

glob_DATETIME = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

def predict_all():
    [predict(*row) for row in tqdm([row for row in walk_dataset()])]# if row[1]=="blue_1"])]# and row[0]=="r_1_0.jpg"])]
    
profiled('predict_all()', globals(), locals())

100%|██████████| 2092/2092 [56:34<00:00,  1.62s/it] 


In [9]:
def calc_overlappings(rects, pred_rects):
    global glob_overlappings    
    overlappings=[(np.max([calc_overlapping(pred_rect,rect, np.max) for pred_rect in pred_rects]) if len(pred_rects) > 0 else 0) for rect in rects]
    print(overlappings)
    glob_overlappings.extend(overlappings)

def remove_margin(rect):
    ((x1,y1),(x2,y2))=rect
    w,h=(abs(x1-x2), abs(y1-y2))
    center=((x1+x2)/2,(y1+y2)/2)
    w_margin=w/12
    h_margin=h/12
    x1+=w_margin
    x2-=w_margin
    y1+=h_margin
    y2-=h_margin
    return ((int(x1),int(y1)),(int(x2),int(y2)))
    
def draw_predicted_rectangles(filename, scene_name, label_resolution, rects, *args):
    global glob_RES
    
    img=imread_resized(scene_name, filename, label_resolution)
    img_rows=glob_RES[np.where((glob_RES[:,0] == scene_name) * (glob_RES[:,1] == filename))]
    
    pred_rects=[]
#     rects=[add_margin(correct_rect_ratio(rect)) for rect in rects]
    for max_row in img_rows:
        [scene_name, filename, scale, (x,y), pred]=max_row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        pred_rect = (x,y), (x + winW_s, y + winH_s)
        pred_rect = remove_margin(pred_rect)
        cv2.rectangle(img, pred_rect[0], pred_rect[1], (0, 255, 0), DRAW_BORDER)
        pred_rects.append(((x,y),(x+winW_s, y+winH_s)))
    for rect in rects: 
        ((x,y),(x2,y2))=rect
        cv2.rectangle(img, (x,y), (x2,y2), (255, 0, 0), DRAW_BORDER)
    
    save_image(img, scene_name, filename, "predicted_labels")
    print(filename)
    calc_overlappings(rects, pred_rects)

glob_overlappings=[]
[draw_predicted_rectangles(*row) for row in tqdm(walk_dataset())]# if row[1]=="blue_1"]# and row[0]=="r_1_0.jpg"]
print("Mean overlapping: ",np.mean(glob_overlappings))

  0%|          | 2/2092 [00:00<03:05, 11.28it/s]

IMG_20200229_075347.jpg
[0.7859541984732824]
IMG_20200229_075348.jpg
[0.6312002389827647]
IMG_20200229_075349.jpg
[0.5761810821889731]


  0%|          | 6/2092 [00:00<03:00, 11.55it/s]

IMG_20200229_075350.jpg
[0.8336842105263158]
IMG_20200229_075353.jpg
[0.11867860851505711]
IMG_20200229_075356.jpg
[0.12267707872103477]


  0%|          | 8/2092 [00:00<02:58, 11.67it/s]

...

0006.jpg
[0.834860493883722]
0007.jpg
[0.7291886195995785]
0008.jpg
[0.8239794749622028]
0009.jpg
[0.7854873932896566]
0010.jpg
[0.13602465603073183]
0011.jpg
[0.7059757792336708]
0012.jpg
[0.1515479056462663]
0013.jpg
[0.7721052631578947]
0014.jpg
[0.8578366654092795]
0015.jpg
[0.9066292247517802]
0016.jpg
[0.7983805668016194]
0017.jpg
[0.8716043123232108]
0018.jpg
[0.881053901850362]
0019.jpg
[0.8583735585947976]
0020.jpg
[0.855430410297667]
0021.jpg
[0.8743362831858407]
0022.jpg
[0.11577708006279434]
0023.jpg
[0.8710646285867525]
0024.jpg
[0.11813125695216908]
0025.jpg
[0.7247787610619469]
0026.jpg
[0.120504712770625]
0027.jpg
[0.7595803164387235]
0028.jpg
[0.1282237032744132]
0029.jpg
[0.13414397952051738]
0030.jpg
[0.7420555108608206]
0031.jpg
[0.8981304778060504]


 99%|█████████▊| 2062/2092 [01:38<00:00, 45.91it/s]

0032.jpg
[0.8521196941502263]
0033.jpg
[0.11430112512656491]
0034.jpg
[0.8490245971162002]
0035.jpg
[0.12770928929394598]
0036.jpg
[0.7475084817642069]
0037.jpg
[0.13105131821633936]
0038.jpg
[0.17641196013289037]
0039.jpg
[0.13376522702104096]
0040.jpg
[0.14056463250011636]
0041.jpg
[0.14432681737800507]
0042.jpg
[0.14265636810279667]
0043.jpg
[0.1476740632067975]
0044.jpg
[0.7232647724059937]
0045.jpg
[0.1147464859437751]
0046.jpg
[0.8261052631578948]
0047.jpg
[0.8147017543859649]
0048.jpg
[0.6536789652247668]
0049.jpg
[0.8171812865497076]
0050.jpg
[0.7625614035087719]
0051.jpg
[0.7541286549707602]
0052.jpg
[0.748]
0053.jpg
[0.8114035087719298]
0054.jpg
[0.7614035087719299]
0055.jpg
[0.7860818713450293]
0056.jpg
[0.7738362573099415]
0057.jpg
[0.8584468538596979]
0058.jpg
[0.1383625907990315]
0059.jpg
[0.6670877192982456]


100%|██████████| 2092/2092 [01:38<00:00, 21.15it/s]

0060.jpg
[0.1385975018190638]
0061.jpg
[0.6969590643274853]
0062.jpg
[0.13639754147272945]
0063.jpg
[0.14300237737737737]
0064.jpg
[0.8570104716893707]
0065.jpg
[0.773978315262719]
0066.jpg
[0.7346081871345029]
0067.jpg
[0.8278635778635779]
0068.jpg
[0.7513450292397661]
0069.jpg
[0.7159649122807018]
0070.jpg
[0.14330721003134797]
0071.jpg
[0.14974777253668764]
0072.jpg
[0.15183672113723928]
0073.jpg
[0.14912495310792517]
0074.jpg
[0.8127760788311247]
0075.jpg
[0.8436660179779446]
0076.jpg
[0.15123645686874534]
Mean overlapping:  0.48582109739794826


In [10]:
#min_pred=0.8 : overlap = 0.623